Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [1]:
%run -i "assignment_helper.py"

Available frameworks:
Forest SDK
Qiskit
D-Wave Ocean


# Measurements

**Exercise 1** (1 point). Measurements in the quantum computers we can access today always measure in the computational basis. This means that either the projection $|0\rangle\langle 0|$ or the $|1\rangle\langle 1|$ is applied on the qubit we are measuring, corresponding to the outcome 0 or 1. The Born rule tells us that we get either of these with the probability of the absolute value of the probability amplitde squared. The qubit afterwards is $|0\rangle$ or $|1\rangle$, respectively.

First, create circuit in your preferred framewrok with one quantum and two classical registers. The object should be called `circuit`.

In [2]:
#
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import BasicAer

q = QuantumRegister(1)
c = ClassicalRegister(2)
circuit = QuantumCircuit(q, c)
circuit.iden(q[0])
#


In [3]:
classical_bits = get_classical_bits(circuit)
assert classical_bits == 2

**Exercise 2** (1 point). Next, extend the circuit with a Hadamard gate and a measurement on the qubit. Write the measurement result in the first classical register. You will get a probabilistic outcome of 0 or 1.

In [6]:
#
backend = BasicAer.get_backend('qasm_simulator')

circuit.h(q[0])
circuit.measure(q[0], c[0])
#job = execute(circuit, backend, shots=100)
#


In [7]:
counts = get_single_measurement_counts(circuit)
assert abs(counts['00']/100-.5) < 0.1

**Exercise 3** (1 point). To see that the quantum state collapses to the basis state indicated by the outcome, apply a second measurement on the same circuit, but write the result in the second register. Executing the program should always give the same outcome in the two subsequent measurements.

In [8]:
#
circuit.measure(q[0], c[1])
#


In [9]:
counts = get_counts(circuit)
assert abs(counts['00']/100-.5) < 0.1
assert abs(counts['11']/100-.5) < 0.1
assert sum(counts.values()) == 100

# Measuring multiqubit systems

**Exercise 4** (1 point). We typically work with local measurements, meaning that the qubits are measured separately. Create a two-qubit circuit with measurements included that reproduces the uniform distribution on four outcomes.

In [15]:
#
q = QuantumRegister(2)
c = ClassicalRegister(2)
circuit = QuantumCircuit(q, c)
circuit.h(q[0])
circuit.h(q[1])
circuit.measure(q[0], c[0])
circuit.measure(q[1], c[1])
#


In [16]:
counts = get_counts(circuit)
assert abs(counts['00']/100-.25) < 0.1
assert abs(counts['01']/100-.25) < 0.1
assert abs(counts['11']/100-.25) < 0.1
assert sum(counts.values()) == 100

**Exercise 5** (1 point). This is a typical product state, showing no correlation between the qubits. In contrast, if you make simultaneous measurements on an entangled state, for instance the $|\phi^+\rangle$ or the $|\phi^-\rangle$ state, the measurement outcomes will be correlated. Create either of these states and measure both qubits.

In [19]:
#
q = QuantumRegister(2)
c = ClassicalRegister(2)
circuit = QuantumCircuit(q, c)
circuit.h(q[0])
circuit.cx(q[0], q[1])
circuit.measure(q, c)
#


In [20]:
counts = get_counts(circuit)
assert abs(counts['00']/100-.5) < 0.1
assert abs(counts['11']/100-.5) < 0.1
assert sum(counts.values()) == 100

Notice that we only observe 00 and 11 as outcomes, even though we made measurements on two spatially separated qubits. If one measurement gives a value, the other one always give the same. 

# Mixed states

**Exercise 6** (1 point). The density matrix formalism is critical in understanding noise models and decoherence, so it is indispensible to start working with current and near-future quantum computers. Many simulator backends provide noise models, but it is out of scope for this course to develop a deep understanding of it.

Instead, let's use the wavefunction simulator to create the (maximally) entangled state $|\phi^+\rangle$. The object should be called circuit and do not include measurements. Your circuit should be in an object called `circuit`.

In [21]:
#
q = QuantumRegister(2)
c = ClassicalRegister(2)
circuit = QuantumCircuit(q, c)
circuit.h(q[0])
circuit.cx(q[0], q[1])
#


In [22]:
amplitudes = get_amplitudes(circuit)
assert all(np.isclose(np.array([np.sqrt(2)/2, 0, 0, np.sqrt(2)/2]), amplitudes))

**Exercise 7** (1 point). Create the density matrix called `dm` of the probability amplitudes contained in the `amplitudes` array. Recall that you need a ket and a bra in this order to produce it, but the shape of the `amplitudes` array is incorrect for transposition. Reshape it first

In [47]:
#
ψ = get_amplitudes(circuit)

ψ_ket =  np.reshape(ψ, (4,1))

dm = ψ_ket.dot(ψ_ket.T.conj())
#print(dm)
#


In [48]:
#
# AUTOGRADER TEST - DO NOT REMOVE
#


Taking the partial trace of a density matrix is the equivalent of taking the marginal of a joint probability distribution along one random variable. If we take the partial trace of this density matrix in any of the qubit subsystems, we are going to get the maximally mixed state, that is, the uniform distribution over the remaining system. This reflects the strong correlation between the two subsystems: if we marginalize over one, we have no predictive power over the other one. We exploit this property when we create protocols for preparing thermal states.